# AUTOENCODERS 

Codes are taken from Aurélien Géron's Hands-On Machine Learning with Scikit-Learn and TensorFlow book but I added some small changes.

In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

## Linear Autoencoder

In [7]:
n_inputs = 129 **2 # input is flatten version of 129x129 matrix
n_hidden = 100 # hidden layer with 100 neurons
n_outputs = n_inputs

learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = fully_connected(X, n_hidden, activation_fn=None)
outputs = fully_connected(hidden, n_outputs, activation_fn=None)

reconstruction_loss = tf.reduce_sum(tf.square(outputs - X)) # MSE

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()

n_iterations = 500 # Number of iterations
codings = hidden # the output of the hidden layer provides the codings

In [8]:
def LinearAutoencoder(X_train):
    with tf.Session() as sess: 
        init.run()
        for iteration in range(n_iterations):
            training_op.run(feed_dict={X: X_train}) # no labels (unsupervised)
        # Test on the same protein
        codings_val = codings.eval(feed_dict={X: X_train})

## Stacked Autoencoder

In [9]:
def StackedAutoencoder(X_train):
    n_inputs = 129 * 129 # for pair distance matrix 
    n_hidden1 = 1000
    n_hidden2 = 500 # codings 
    n_hidden3 = n_hidden1 
    n_outputs = n_inputs

    learning_rate = 0.01
    l2_reg = 0.001

    X = tf.placeholder(tf.float32, shape=[None, n_inputs]) 

    with tf.contrib.framework.arg_scope(
        [fully_connected], activation_fn=tf.nn.elu, weights_initializer=tf.contrib.layers.variance_scaling_initializer(),
        weights_regularizer=tf.contrib.layers.l2_regularizer(l2_reg)):

        hidden1 = fully_connected(X, n_hidden1)
        hidden2 = fully_connected(hidden1, n_hidden2) # codings
        hidden3 = fully_connected(hidden2, n_hidden3)
        outputs = fully_connected(hidden3, n_outputs, activation_fn=None)

    reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) # MSE
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = reconstruction_loss + reg_losses

    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

    init = tf.global_variables_initializer()

    n_epochs = 5
    batch_size = 150
    
    with tf.Session() as sess: 
        init.run() 
        for epoch in range(n_epochs):
            batch_start = 0 # starting index of the batch
            n_batches = len(X_train) // batch_size
            for iteration in range(n_batches):
                X_batch = X_train[batch_start : batch_start+batch_size]
                start += batch_size
                sess.run(training_op, feed_dict={X: X_batch})

##  Stacked Autoencoder with Tied Weights

This code is fairly straightforward, but there are a few important things to note:

    • First,weight3andweights4arenotvariables,theyarerespectivelythetranspose of weights2 and weights1 (they are “tied” to them).

    • Second, since they are not variables, it’s no use regularizing them: we only regula‐ rize weights1 and weights2.

    • Third, biases are never tied, and never regularized.

In [22]:
def StackedAutoencoderWithTiedWeights(X_train):
    n_inputs = 129 * 129 # for pair distance matrix 
    n_hidden1 = 1000
    n_hidden2 = 500 # codings 
    n_hidden3 = n_hidden1 
    n_outputs = n_inputs

    learning_rate = 0.01
    l2_reg = 0.001

    activation = tf.nn.elu
    regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
    initializer = tf.contrib.layers.variance_scaling_initializer()

    X = tf.placeholder(tf.float32, shape=[None, n_inputs])

    weights1_init = initializer([n_inputs, n_hidden1])
    weights2_init = initializer([n_hidden1, n_hidden2])

    weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1") 
    weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2") 
    weights3 = tf.transpose(weights2, name="weights3") # tied weights
    weights4 = tf.transpose(weights1, name="weights4") # tied weights

    biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
    biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
    biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
    biases4 = tf.Variable(tf.zeros(n_outputs), name="biases4")

    hidden1 = activation(tf.matmul(X, weights1) + biases1)
    hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
    hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
    outputs = tf.matmul(hidden3, weights4) + biases4

    reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))
    reg_loss = regularizer(weights1) + regularizer(weights2)

    loss = reconstruction_loss + reg_loss

    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

    init = tf.global_variables_initializer()
    
    with tf.Session() as sess: 
        init.run() 
        for epoch in range(n_epochs):
            batch_start = 0 # starting index of the batch
            n_batches = len(X_train) // batch_size
            for iteration in range(n_batches):
                X_batch = X_train[batch_start : batch_start+batch_size]
                start += batch_size
                sess.run(training_op, feed_dict={X: X_batch})
                
        